This notebook contains code that tests the geospatial files and libraries and see if we can make them work with D-REPR

**Setup notebook**: you should use a virtual environment that GDAL is installed inside (an `environment.yml` is provided so you can use it instead of creating your own)

In [1]:
import matplotlib.pyplot as plt, rdflib, pandas as pd, numpy as np, sys, os, random, math, fiona
from osgeo import gdal, osr, gdal_array
from collections import defaultdict, Counter
from dotenv import load_dotenv
from tqdm.auto import tqdm
from typing import *

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

# next cell
%reload_ext autoreload

In [2]:
load_dotenv(verbose=True)
paths = ["../../"]
for path in paths:
    if path not in sys.path:
        sys.path.insert(0, path)
        
from drepr import __version__, DRepr, outputs
from raster import *
print("drepr version:", __version__)

drepr version: 2.7


**working with shapefile**

In [3]:
shpfile = "./data/ilubabor-dega/ilubabor-dega.shp"

using fiona to load and see what is in the shapefile

using drepr to map the file

In [4]:
dsmodel = DRepr.parse_from_file("./woreda_shapefile.yml")
shp_sm = outputs.ArrayBackend.from_drepr(dsmodel, shpfile)

In [27]:
for c in shp_sm.c("mint:Place"):
    for record in c.iter_records():
        print("place", record.to_dict())
        print("bounding", shp_sm.get_record_by_id(record.s('mint-geo:bounding')).to_dict())
        place = record

place {'@id': (0,), 'mint:region': ['Oromia'], 'mint:zone': ['Ilubabor'], 'mint:district': ['Dega'], 'mint-geo:bounding': [(0,)]}
bounding {'@id': (0,), 'rdf:value': [array([[[36.24299932,  8.50049719],
        [36.23500927,  8.4954872 ],
        [36.20864912,  8.48833724],
        [36.18917901,  8.48862727],
        [36.1657089 ,  8.49581731],
        [36.1409988 ,  8.51360735],
        [36.11778869,  8.51733738],
        [36.08883853,  8.51430743],
        [36.05888838,  8.51958747],
        [36.03462828,  8.53830752],
        [36.00157812,  8.55347757],
        [35.99509809,  8.55272758],
        [35.98467804,  8.5600376 ],
        [35.98164802,  8.56215761],
        [35.98146803,  8.56766761],
        [35.98179805,  8.57822761],
        [35.98236807,  8.58968761],
        [35.97741806,  8.60373762],
        [35.96547802,  8.61279765],
        [35.95858799,  8.61790766],
        [35.94866074,  8.62358297],
        [35.98727919,  8.6622011 ],
        [36.04415038,  8.66222492],
     

**working with raster dataset:** cropping a weather dataset (GLDAS or GPM)

In [5]:
weather_dataset = {
    "gldas": {"repr": "./gldas.yml", "data": "GLDAS_NOAH025_3H.A20080101.0000.021.nc4"},
    "gpm": {"repr": "./gpm.yml", "data": "3B-MO.MS.MRG.3IMERG.20080101-S000000-E235959.01.V06B.HDF5.nc4"},
}['gldas']
variable = "atmosphere_water__precipitation_mass_flux"

load dataset using drepr

In [7]:
from drepr.engine import complete_description

In [9]:
dsmodel = DRepr.parse_from_file(weather_dataset['repr'])
sm = outputs.ArrayBackend.from_drepr(dsmodel, weather_dataset['data'])

crop from a given shape file

In [21]:
mint = sm.ns("https://mint.isi.edu/")
rdf = sm.ns(outputs.Namespace.RDF)
mint_geo = sm.ns("https://mint.isi.edu/geo")

rasters = []

for c in sm.c(mint.Variable).filter(outputs.FCondition(mint.standardName, "==", variable)):
    for raster_id, sc in c.group_by(mint_geo.raster):
        data = sc.p(rdf.value).as_ndarray([sc.p(mint_geo.lat), sc.p(mint_geo.long)])
        gt_info = sm.get_record_by_id(raster_id)
        gt = GeoTransform(x_0=gt_info.s(mint_geo.x_0),
                          y_0=gt_info.s(mint_geo.y_0),
                          dx=gt_info.s(mint_geo.dx), dy=gt_info.s(mint_geo.dy))
        raster = Raster(data.data, gt, int(gt_info.s(mint_geo.epsg)),
               data.nodata.value if data.nodata is not None else None)
        raster.to_geotiff("./debug.tif")
        raster = raster.crop(vector_file=shpfile, resampling_algo=ReSample.BILINEAR)
        raster.to_geotiff("./cropped_debug.tif")

        rasters.append(raster)

**create new in-memory dataset:** useful if we want to create d-repr dataset from some data in-memory. We are going to use it to demonstrate the process of cropping a dataset based on a bounding polygon and return it as a d-repr dataset

inject in memory first

In [30]:
from drepr.executors.readers.reader_container import ReaderContainer
from drepr.executors.readers.np_dict import NPDictReader

for i, r in enumerate(rasters):
    reader = NPDictReader({
        "variable": r.data,
        "nodata": r.nodata,
        "gt_x_0": r.geotransform.x_0,
        "gt_y_0": r.geotransform.y_0,
        "gt_dx": r.geotransform.dx,
        "gt_dy": r.geotransform.dy,
        "gt_epsg": 4326,
        "gt_x_slope": r.geotransform.x_slope,
        "gt_y_shope": r.geotransform.y_slope,
        "place_region": place.s("mint:region"),
        "place_zone": place.s("mint:zone"),
        "place_district": place.s("mint:district")
    })
    ReaderContainer.get_instance().set(f"resource-{i}", reader)

create drepr mapping for them

In [36]:
dsmodel = {
    "version": "2",
    "resources": "container",
    "attributes": {
        "variable": "$.variable[:][:]",
        "nodata": "$.nodata",
        "gt_x_0": "$.gt_x_0",
        "gt_y_0": "$.gt_y_0",
        "gt_dx": "$.gt_dx",
        "gt_dy": "$.gt_dy",
        "gt_epsg": "$.gt_epsg",
        "gt_x_slope": "$.gt_x_slope",
        "gt_y_shope": "$.gt_y_shope",
        "place_region": "$.place_region",
        "place_zone": "$.place_zone",
        "place_district": "$.place_district",
    },
    "alignments": [
        {"type": "dimension", "source": "variable", "target": x, "aligned_dims": []}
        for x in ["nodata", "gt_x_0", "gt_y_0", "gt_dx", "gt_dy", "gt_epsg", "gt_x_slope", "gt_y_shope", "place_region", "place_zone", "place_district"]
    ],
    "semantic_model": {
        "mint:Variable:1": {
            "properties": [
                ("rdf:value", "variable")
            ],
            "links": [
                ("mint:place", "mint:Place:1"),
                ("mint-geo:raster", "mint-geo:Raster:1")
            ]
        },
        "mint-geo:Raster:1": {
            "properties": [
                ("mint-geo:x_0", "gt_x_0"),
                ("mint-geo:y_0", "gt_y_0"),
                ("mint-geo:dx", "gt_dx"),
                ("mint-geo:dy", "gt_dy"),
                ("mint-geo:epsg", "gt_epsg"),
                ("mint-geo:x_slope", "gt_x_slope"),
                ("mint-geo:y_shope", "gt_y_shope"),
            ]
        },
        "mint:Place:1": {
            "properties": [
                ("mint:region", "place_region"),
                ("mint:zone", "place_zone"),
                ("mint:district", "place_district"),
            ]
        },
        "prefixes": {
            "mint": "https://mint.isi.edu/",
            "mint-geo": "https://mint.isi.edu/geo"
        }
    }
}

dsmodel = DRepr.parse(dsmodel)

now create N-datasets from N rasters (resources)

In [37]:
for i in range(len(rasters)):
    new_sm = outputs.ArrayBackend.from_drepr(dsmodel, f"resource-{i}")

In [44]:
for c in new_sm.iter_classes():
    print(">>>", c.id)
    for r in c.iter_records():
        print(r.to_dict())

>>> mint:Variable:1
{'@id': (0, 0), 'rdf:value': [0.04891601], 'mint:place': [()], 'mint-geo:raster': [()]}
{'@id': (0, 1), 'rdf:value': [0.04556715], 'mint:place': [()], 'mint-geo:raster': [()]}
{'@id': (0, 2), 'rdf:value': [0.0484128], 'mint:place': [()], 'mint-geo:raster': [()]}
{'@id': (0, 3), 'rdf:value': [0.035574928], 'mint:place': [()], 'mint-geo:raster': [()]}
{'@id': (1, 0), 'rdf:value': [0.06609181], 'mint:place': [()], 'mint-geo:raster': [()]}
{'@id': (1, 1), 'rdf:value': [0.05395234], 'mint:place': [()], 'mint-geo:raster': [()]}
{'@id': (1, 2), 'rdf:value': [0.0533901], 'mint:place': [()], 'mint-geo:raster': [()]}
{'@id': (1, 3), 'rdf:value': [0.03947892], 'mint:place': [()], 'mint-geo:raster': [()]}
>>> mint-geo:Raster:1
{'@id': (0,), 'mint-geo:x_0': [35.900000000000006], 'mint-geo:y_0': [8.700000000000003], 'mint-geo:dx': [0.10000000000000142], 'mint-geo:dy': [-0.10000000000000142], 'mint-geo:epsg': [4326], 'mint-geo:x_slope': [0.0], 'mint-geo:y_shope': [0.0]}
>>> mint:P